In [17]:
import os

import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=15,10
print("Matplotlib:{}".format(mpl.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

import keras
print("Keras:{}".format(keras.__version__))

current_path = os.getcwd()
base_dir = os.path.dirname(current_path)

if not base_dir in sys.path:
    sys.path.append(base_dir)

print(sys.path)

%reload_ext autoreload
%autoreload 2
import datasetslib

datasetslib.datasets_root = os.path.join(os.path.expanduser('~'),'datasets')

from tensorflow.examples.tutorials.mnist import input_data

dataset_home = os.path.join(datasetslib.datasets_root, 'mnist')
mnist = input_data.read_data_sets(dataset_home, one_hot=False)

X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels

pixel_size = 28

while True:
    train_images, train_labels = mnist.train.next_batch(4)
    if len(set(train_labels)) == 4:
        break
    
while True:
    test_images, test_labels = mnist.test.next_batch(4)
    if len(set(test_labels)) == 4:
        break
        
learning_rate = 0.001
n_epochs = 20
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

n_inputs = 784
n_outputs = n_inputs

x = tf.placeholder(dtype=tf.float32, name='x', shape=[None, n_inputs])
y = tf.placeholder(dtype=tf.float32, name='y', shape=[None, n_outputs])

n_layers = 2
n_neurons = [512, 256]

n_neurons.extend(list(reversed(n_neurons)))
n_layers = n_layers * 2

w = []
b = []

for i in range(n_layers):
    w.append(tf.Variable(tf.random_normal([n_inputs if i==0 else n_neurons[i-1], n_neurons[i]]), 
              name='w_{0:04d}'.format(i)))
    b.append(tf.Variable(tf.zeros([n_neurons[i]]), 
             name='b_{0:04d}'.format(i)))
w.append(tf.Variable(tf.random_normal([n_neurons[n_layers-1] if n_layers>0 else n_inputs, n_outputs]), 
              name='w_out'))
b.append(tf.Variable(tf.zeros([n_outputs]), name='b_out'))

layer = x

for i in range(n_layers):
    layer = tf.nn.sigmoid(tf.matmul(layer, w[i]) + b[i])
layer = tf.nn.sigmoid(tf.matmul(layer, w[n_layers]) + b[n_layers])
model = layer

mse = tf.losses.mean_squared_error
loss = mse(predictions=model, labels=y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer = optimizer.minimize(loss)

with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        for batch in range(n_batches):
            X_batch, _ = mnist.train.next_batch(batch_size)
            feed_dict = {x: X_batch, y: X_batch}
            _, batch_loss = tfs.run([optimizer, loss], feed_dict)
            epoch_loss += batch_loss
        if (epoch % 10) == 9  or  (epoch ==0):
            average_loss = epoch_loss / n_batches
            print('epoch: {0:04d} loss = {1:0.6f}'.format(epoch, average_loss))
    Y_train_pred = tfs.run(model, feed_dict={x: train_images})
    Y_test_pred = tfs.run(model, feed_dict={x: test_images})
    

import random

def display_images(images, labls, count=0, one_hot=False):
    if count == 0:
        count = images.shape[0]
    
    idx_list = random.sample(range(len(labels)), count)
    for i in range(count):
        plt.subplot(4, 4, i+1)
        plt.title(labls[i])
        plt.imshow(images[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    
    

         
          
        
    
    
    

NumPy:1.16.2
Matplotlib:2.2.2
TensorFlow:1.13.1
Keras:2.2.4
['C:\\Work\\pypath', 'c:\\python36\\python36.zip', 'c:\\python36\\DLLs', 'c:\\python36\\lib', 'c:\\python36', '', 'c:\\python36\\lib\\site-packages', 'c:\\python36\\lib\\site-packages\\setuptools-36.5.0-py3.6.egg', 'c:\\python36\\lib\\site-packages\\win32', 'c:\\python36\\lib\\site-packages\\win32\\lib', 'c:\\python36\\lib\\site-packages\\Pythonwin', 'c:\\python36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\wj_di\\.ipython', '../datasetslib', 'C:\\Work\\github\\tensorflow_learning']
Extracting C:\Users\wj_di\datasets\mnist\train-images-idx3-ubyte.gz
Extracting C:\Users\wj_di\datasets\mnist\train-labels-idx1-ubyte.gz
Extracting C:\Users\wj_di\datasets\mnist\t10k-images-idx3-ubyte.gz
Extracting C:\Users\wj_di\datasets\mnist\t10k-labels-idx1-ubyte.gz
epoch: 0000 loss = 0.161680
epoch: 0009 loss = 0.098568
epoch: 0019 loss = 0.086628


In [18]:
display_images(train_images.reshape(-1,pixel_size,pixel_size),train_labels)
display_images(Y_train_pred.reshape(-1,pixel_size,pixel_size),train_labels)

NameError: name 'labels' is not defined

In [ ]:
display_images(test_images.reshape(-1,pixel_size,pixel_size),test_labels)
display_images(Y_test_pred.reshape(-1,pixel_size,pixel_size),test_labels)